In [1]:
# Import the python packages you will need 
import pandas 
import numpy as np
import netCDF4 as nc
import xarray as xr
import xesmf
import time
import warnings
warnings.filterwarnings("ignore")
experiment="no_bias_correction"

In [2]:
# NEP Bering Grid 
bering_nep_grid_file = '/work/role.medgrp/NEP/plotting/Figure_19_20/nep_bering_grid.nc'
grd_fid = nc.Dataset(bering_nep_grid_file)

# extracting lat/lon from geolat/geolon
nep_lat = grd_fid.variables['geolat'][:]
nep_lon = grd_fid.variables['geolon'][:]

#### Interpolation for Eastern and Northern Bering Sea for Kearney spatial plotting

In [3]:
# daily NEP bering sea bottom temp
#nep_fil = f'/work/Utheri.Wagura/NEP/plotting/Figure_19_20/{experiment}/nep_tob_bering_1993-2019_daily_ts.nc'
nep_fil = f'/work/Utheri.Wagura/NEP/plotting/Figure_19_20/{experiment}/nep_tob_bering_1993-2024_daily_ts.nc'
nep_ds = xr.open_dataset(nep_fil)
nep_ds = nep_ds.assign_coords(lon = (('yh', 'xh'), nep_lon))
nep_ds = nep_ds.assign_coords(lat = (('yh', 'xh'), nep_lat))

print(nep_ds)

## daily bottom temp
trawl_fil = '/work/role.medgrp/NEP/TOOLS/coldpool/data/ebs_nbs_temperature_full_area.csv'
df = pandas.read_csv(trawl_fil)

# drop rows that go back for forward in time further than the temporal extent of the NEP hindcast
df = df[df.year >= 1993].reset_index(drop=True)
#df = df[df.year <= 2019].reset_index(drop=True)

# only measurements with haul type 3
df = df[df.haul_type == 3].reset_index(drop=True)

# generate copy of trawl dataframe with added empty column for interpolated NEP temp
df2 = df.assign(nep_tob=np.nan*np.zeros(len(df)))

<xarray.Dataset> Size: 1GB
Dimensions:     (time: 11688, nv: 2, yh: 150, xh: 150)
Coordinates:
  * nv          (nv) float64 16B 1.0 2.0
  * time        (time) datetime64[ns] 94kB 1993-01-01T12:00:00 ... 2024-12-31...
  * xh          (xh) float64 1kB 233.6 233.6 233.7 233.8 ... 244.5 244.6 244.6
  * yh          (yh) float64 1kB 51.52 51.57 51.62 51.67 ... 57.56 57.59 57.62
    lon         (yh, xh) float32 90kB 195.4 195.5 195.6 ... 185.3 185.4 185.5
    lat         (yh, xh) float32 90kB 50.08 50.15 50.23 ... 68.17 68.25 68.33
Data variables:
    average_DT  (time) timedelta64[ns] 94kB ...
    average_T1  (time) datetime64[ns] 94kB ...
    average_T2  (time) datetime64[ns] 94kB ...
    time_bnds   (time, nv) datetime64[ns] 187kB ...
    tob         (time, yh, xh) float32 1GB ...
Attributes:
    NumFilesInSet:       1
    title:               NEP10_PHYSICS_decr7_e1
    associated_files:    areacello: 19900101.ocean_static.nc
    grid_type:           regular
    grid_tile:           N/A
  

In [4]:
# convert time to ALASKA time
times = np.array(df['start_time'].to_list(), dtype="datetime64")
times_alaska = times + np.timedelta64(8,"h")

trawl_locs = xr.Dataset()
trawl_locs['lon'] = xr.DataArray(data=df['longitude'].to_list(), dims=("trawl_sample") )
trawl_locs['lat'] = xr.DataArray(data=df['latitude'].to_list(), dims=("trawl_sample") )
print(trawl_locs)

 # Define regridder
nep_regridder = xesmf.Regridder(nep_ds, trawl_locs , 'bilinear', locstream_out=True)

# Interpolate times
nep_ds_interp = nep_ds.interp(time=times_alaska)

# Regrid
nep_regrided = nep_regridder(nep_ds_interp)

print(nep_regrided)

for i in range( len(df2)):
    df2['nep_tob'][i] = nep_regrided.tob.isel(time=i,trawl_sample=i)
 
new_csv = f'/work/Utheri.Wagura/NEP/plotting/Figure_19_20/{experiment}/ebs_nbs_temperature_full_area_nep.csv'
df2.to_csv(new_csv)
df2['nep_tob']

<xarray.Dataset> Size: 195kB
Dimensions:  (trawl_sample: 12193)
Dimensions without coordinates: trawl_sample
Data variables:
    lon      (trawl_sample) float64 98kB -159.8 -159.7 -159.7 ... -175.7 -175.5
    lat      (trawl_sample) float64 98kB 56.67 57.01 57.34 ... 60.0 59.34 58.67
<xarray.Dataset> Size: 1GB
Dimensions:  (time: 12193, trawl_sample: 12193, nv: 2)
Coordinates:
  * nv       (nv) float64 16B 1.0 2.0
  * time     (time) datetime64[s] 98kB 1993-06-06T15:00:00 ... 1998-07-29T23:...
Dimensions without coordinates: trawl_sample
Data variables:
    tob      (time, trawl_sample) float64 1GB 5.362 4.675 3.831 ... 2.695 3.38
Attributes:
    regrid_method:  bilinear


0        5.361503
1        4.681043
2        3.855108
3        3.595562
4        4.246317
           ...   
12188    1.836423
12189    2.169506
12190    1.980478
12191    2.667769
12192    3.380377
Name: nep_tob, Length: 12193, dtype: float64

#### Interpolation for Index Bering Sea for Rohan CPA index processing and analysis

In [5]:
## daily bottom temp
trawl_fil = '/work/role.medgrp/NEP/TOOLS/coldpool/data/index_hauls_temperature_data.csv'
df = pandas.read_csv(trawl_fil)
# drop rows that go back for forward in time further than the temporal extent of the NEP hindcast
df = df[df.year >= 1993].reset_index(drop=True)
#df = df[df.year <= 2019].reset_index(drop=True)

# append fake 2020 time values for nep regridding
df_2020 = df[df.year==2019].reset_index(drop=True)
df_2020.year = 2020
df_2020.gear_temperature = np.nan
df_2020.surface_temperature  = np.nan
df_2020.cruise = np.nan
for row in df_2020.itertuples():
    index=row[0]
    time_str = df_2020.start_time[index].split('2019')[1] 
    df_2020.start_time[index] = ('2020' + time_str)

df = pandas.concat([df, df_2020]).reset_index(drop=True)

nep_fil = f'/work/Utheri.Wagura/NEP/plotting/Figure_19_20/{experiment}/nep_tob_bering_1993-2024_daily_ts.nc'
#nep_fil = f'/work/Utheri.Wagura/NEP/plotting/Figure_19_20/{experiment}/nep_tob_bering_1993-2019_daily_ts.nc'
nep_ds = xr.open_dataset(nep_fil)
nep_ds = nep_ds.assign_coords(lon = (('yh', 'xh'), nep_lon))
nep_ds = nep_ds.assign_coords(lat = (('yh', 'xh'), nep_lat))

# generate copy of trawl dataframe with added empty column for interpolated NEP temp
df2 = df.assign(nep_tob=np.nan*np.zeros(len(df)))

trawl_locs = xr.Dataset()
# get lat lon values for regridding
trawl_locs['lon'] = xr.DataArray(data=df['longitude'].to_numpy(), dims=('trawl_sample'))
trawl_locs['lat'] = xr.DataArray(data=df['latitude'].to_numpy(), dims=('trawl_sample'))

# Get time values for time interpolation
print("Creating array of Alaska times:")
times = np.array(df['start_time'].to_list(), dtype="datetime64")
times_alaska = times + np.timedelta64(8,"h")

# Create regridder to interp latlon to to trawl locations using xesmf
print("Creating regridder to trawl locations:")
nep_regridder = xesmf.Regridder(nep_ds, trawl_locs, 'bilinear', locstream_out=True)

# Interpolate time to Alaska Daylight Time: UTC-8
print("Performing interpolation: ")
nep_interpreted = nep_ds.interp(time=times_alaska)
print("Interpolated times: ")

# regrid NEP and GLORYS bottom temperature value
print("Regridding to trawl locations:")
nep_regridded = nep_regridder(nep_interpreted)
print("Regrided dataset:")

# Write interpreted values to dataframe
print("Writing interpolated values to dataframe")
for i in range( len(df2) ):
    df2['nep_tob'][i] = nep_regridded.tob.isel(time=i,trawl_sample=i)

print("Saving dataframe")
new_csv = f'/work/Utheri.Wagura/NEP/plotting/Figure_19_20/{experiment}/index_hauls_temperature_data_nep.csv'
df2.to_csv(new_csv)
df2['nep_tob']

Creating array of Alaska times:
Creating regridder to trawl locations:
Performing interpolation: 
Interpolated times: 
Regridding to trawl locations:
Regrided dataset:
Writing interpolated values to dataframe
Saving dataframe


0        5.361503
1        4.681043
2        3.855108
3        3.595562
4        4.246317
           ...   
11654    1.502212
11655    1.393018
11656    1.043741
11657    0.341571
11658    0.901552
Name: nep_tob, Length: 11659, dtype: float64